![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/ONNX_Configs_in_Spark_NLP_AlbertForQuestionAnswering.ipynb)

## Setting ONNX configs in SparkNLP

Starting from Spark NLP 5.1.2, you can configure ONNX-related settings within your Spark session. This allows you to fine-tune the behavior of the ONNX engine for your specific needs.

Here are the available options for CPU:

- **intraOpNumThreads**: This setting, `spark.jsl.settings.onnx.intraOpNumThreads`, controls the number of threads used for intra-operation parallelism when executing ONNX models. You can set this value to optimize the performance of ONNX execution. To understand how this affects your ONNX tasks, refer to the ONNX documentation.

- **optimizationLevel**: Use `spark.jsl.settings.onnx.optimizationLevel` to specify the optimization level for ONNX execution. This setting influences how aggressively Spark NLP optimizes the execution of ONNX models. Explore the available options to determine which level suits your workload best in [this ONNX documentation]((https://onnxruntime.ai/docs/api/java/ai/onnxruntime/OrtSession.SessionOptions.OptLevel.html)).

- **executionMode**: With `spark.jsl.settings.onnx.executionMode`, you can choose the execution mode for ONNX models. Different modes may offer trade-offs between performance and resource utilization. Review the available options to select the mode that aligns with your requirements in [this ONNX documentation]((https://onnxruntime.ai/docs/api/java/ai/onnxruntime/OrtSession.SessionOptions.ExecutionMode.html))


Here are the available options for CUDA:

- **gpuDeviceId**: Use `spark.jsl.settings.onnx.gpuDeviceId` to define the GPU device to execute on

To find more information and detailed usage instructions for these ONNX configuration options, refer to the [ONNX API documentation](https://onnxruntime.ai/docs/api/).

In [1]:
# Let's set our config based on our needs:
onnx_params = {
    "spark.jsl.settings.onnx.intraOpNumThreads": "5",
    "spark.jsl.settings.onnx.optimizationLevel": "BASIC_OPT",
    "spark.jsl.settings.onnx.executionMode": "SEQUENTIAL"
}

## Import AlbertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `AlbertForQuestionAnswering` is only available since in `Spark NLP 5.1.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import ALBERT models trained/fine-tuned for question answering via `AlbertForQuestionAnswering`. These models are usually under `Question Answering` category and have `albert` in their labels
- Reference: [TFAlbertForQuestionAnswering](https://huggingface.co/transformers/model_doc/albert#transformers.TFAlbertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=albert&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [twmkn9/albert-base-v2-squad2](https://huggingface.co/twmkn9/albert-base-v2-squad2) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.
- In addition to the ALBERT model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [13]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

MODEL_NAME = "twmkn9/albert-base-v2-squad2"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at twmkn9/albert-base-v2-squad2 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

('onnx_models/twmkn9/albert-base-v2-squad2/tokenizer_config.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/special_tokens_map.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/spiece.model',
 'onnx_models/twmkn9/albert-base-v2-squad2/added_tokens.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [14]:
!ls -l {EXPORT_PATH}

total 46752
-rw-r--r-- 1 root root      793 Jun 17 22:30 config.json
-rw-r--r-- 1 root root 44817937 Jun 17 22:30 model.onnx
-rw-r--r-- 1 root root      286 Jun 17 22:30 special_tokens_map.json
-rw-r--r-- 1 root root   760289 Jun 17 22:30 spiece.model
-rw-r--r-- 1 root root     2616 Jun 17 22:30 tokenizer_config.json
-rw-r--r-- 1 root root  2275547 Jun 17 22:30 tokenizer.json


- As you can see, we need to move `spiece.model` from the tokenizer to `assets` folder which Spark NLP will look for

In [15]:
!mkdir {EXPORT_PATH}/assets

In [16]:
!mv {EXPORT_PATH}/spiece.model {EXPORT_PATH}/assets

Voila! We have our `spiece.model` inside assets directory

In [17]:
!ls -lR {EXPORT_PATH}

onnx_models/twmkn9/albert-base-v2-squad2:
total 46012
drwxr-xr-x 2 root root     4096 Jun 17 22:30 assets
-rw-r--r-- 1 root root      793 Jun 17 22:30 config.json
-rw-r--r-- 1 root root 44817937 Jun 17 22:30 model.onnx
-rw-r--r-- 1 root root      286 Jun 17 22:30 special_tokens_map.json
-rw-r--r-- 1 root root     2616 Jun 17 22:30 tokenizer_config.json
-rw-r--r-- 1 root root  2275547 Jun 17 22:30 tokenizer.json

onnx_models/twmkn9/albert-base-v2-squad2/assets:
total 744
-rw-r--r-- 1 root root 760289 Jun 17 22:30 spiece.model


## Import and Save AlbertForQuestionAnswering in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 40.9 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function with the ONNX parameters that best fit our needs:

In [9]:
import sparknlp

spark = sparknlp.start(params=onnx_params)

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `AlbertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [18]:
from sparknlp.annotator import AlbertForQuestionAnswering

spanClassifier = AlbertForQuestionAnswering.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [19]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [20]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your AlbertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [21]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 44524
-rw-r--r-- 1 root root 44824911 Jun 17 22:32 albert_classification_onnx
-rw-r--r-- 1 root root   760289 Jun 17 22:32 albert_spp
drwxr-xr-x 2 root root     4096 Jun 17 22:32 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [22]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = AlbertForQuestionAnswering.load(f"./{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([[
    "What's my name?",
    "My name is Clara and I live in Berkeley."
]]).toDF("question", "context")

result = pipeline.fit(example).transform(example)
result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `AlbertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
